In [15]:
import tensorflow as tf

# List all physical devices
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f"Number of GPUs available: {len(gpus)}")
    for gpu in gpus:
        print(f"GPU Name: {gpu}")
        # Get detailed information about the GPU
        details = tf.config.experimental.get_device_details(gpu)
        for key, value in details.items():
            print(f"{key}: {value}")
else:
    print("No GPU found.")

import subprocess

def get_gpu_memory():
    try:
        # Run nvidia-smi to get GPU memory info
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.total', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        # Split the result into lines and convert to integer
        gpu_memory = result.strip().split('\n')
        gpu_memory = [int(mem) for mem in gpu_memory]
        return gpu_memory
    except Exception as e:
        print(f"Error: {e}")
        return None

gpu_memory = get_gpu_memory()
if gpu_memory:
    for i, mem in enumerate(gpu_memory):
        print(f"GPU {i} Total memory: {mem / 1024:.2f} GB")
else:
    print("No GPU found or nvidia-smi not available.")

Number of GPUs available: 1
GPU Name: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
device_name: NVIDIA GeForce RTX 4090
compute_capability: (8, 9)
GPU 0 Total memory: 23.99 GB


In [16]:
import os
import numpy as np
import soundfile as sf
from tqdm import tqdm
import random
from spafe.features.mfcc import mfcc
from spafe.features.gfcc import gfcc
from spafe.features.lfcc import lfcc
from spafe.features.lpc import lpc
from spafe.features.msrcc import msrcc
from spafe.features.ngcc import ngcc
from spafe.features.pncc import pncc
from spafe.features.psrcc import psrcc
from spafe.features.rplp import rplp

In [17]:
def extract_feature(audio_file, feature_func, feature_name):
    # Load audio file
    signal, sample_rate = sf.read(audio_file)

    # Extract feature
    feature = feature_func(signal, fs=sample_rate)

    # Save feature to numpy file
    np.save(f"{os.path.splitext(audio_file)[0]}_{feature_name}.npy", feature)

if __name__ == "__main__":
    audio_file = "'F:\Awais_data\Datasets\Halftruth\HAD_train\train\conbine\*.wav'"

    # Extract and save each feature
    features = {
        "gfcc": gfcc,
        "lfcc": lfcc,
        "lpc": lpc,
        "mfcc": mfcc,
        "msrcc": msrcc,
        "ngcc": ngcc,
        "pncc": pncc,
        "psrcc": psrcc,
        "rplp": rplp,
    }

HalfTruth

In [10]:

def extract_feature(audio_file, feature_func, max_frames):
    # Load audio file to get sample rate
    signal, sample_rate = sf.read(audio_file)

    # Load audio file up to 4 seconds
    signal = signal[:int(4 * sample_rate)]

    # Extract feature
    feature = feature_func(signal, fs=sample_rate)

    # Pad or truncate feature to fixed length
    if feature.shape[0] < max_frames:
        feature = np.pad(feature, ((0, max_frames - feature.shape[0]), (0, 0)), mode='constant')
    elif feature.shape[0] > max_frames:
        feature = feature[:max_frames, :]

    return feature

def read_labels(labels_file):
    labels_dict = {}
    with open(labels_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            audio_name = parts[0]
            label = int(parts[-1])  # Assuming the label is at the last index
            labels_dict[audio_name] = label
    return labels_dict

if __name__ == "__main__":
    audio_path = "F:\\Awais_data\\Datasets\\Halftruth\\HAD_train\\train\\conbine"
    labels_file = "F:\Awais_data\Datasets\Halftruth\HAD_train\HAD_train_label.txt"
    output_feature_file = "F:\Awais_data\Datasets\Halftruth\Features\Training\HalfTruth_train_mfcc_features.npy"
    output_labels_file = "F:\Awais_data\Datasets\Halftruth\Features\Training\HalfTruth_train_mfcc_labels.npy"
    max_frames = 40  # Maximum number of frames to consider

    # Read labels
    labels_dict = read_labels(labels_file)

    # Extract features and store labels
    features_list = []
    labels_list = []

    # Iterate over the labeled audio files
    for audio_name, label in tqdm(labels_dict.items(), desc="Extracting features"):
        audio_file = os.path.join(audio_path, audio_name + ".wav")
        # Extract feature
        feature = extract_feature(audio_file, mfcc, max_frames)
        # Store feature and label
        features_list.append(feature)
        labels_list.append(label)
        # Print label
        # print(f"File: {audio_name}, Label: {'spoof' if label == 0 else 'bonafide'}")

    # Convert lists to numpy arrays
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    # Save features and labels to numpy files
    np.save(output_feature_file, features_array)
    np.save(output_labels_file, labels_array)

Extracting features: 100%|███████████████████████████████████████████████████████| 53093/53093 [13:01<00:00, 67.94it/s]


ASVspoof2019-LA

In [29]:
def extract_feature(audio_file, feature_func, max_frames):
    # Load audio file to get sample rate
    signal, sample_rate = sf.read(audio_file)

    # Load audio file up to 4 seconds
    signal = signal[:int(4 * sample_rate)]

    # Extract feature
    feature = feature_func(signal, fs=sample_rate)

    # Pad or truncate feature to fixed length
    if feature.shape[0] < max_frames:
        feature = np.pad(feature, ((0, max_frames - feature.shape[0]), (0, 0)), mode='constant')
    elif feature.shape[0] > max_frames:
        feature = feature[:max_frames, :]

    return feature

def read_labels(labels_file):
    labels_dict = {}
    with open(labels_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            audio_name = parts[1]
            label = parts[4]  # Assuming the label is at the 5th index
            # print(label)
            label = 1 if label == 'spoof' else 0
            labels_dict[audio_name] = label
    return labels_dict

if __name__ == "__main__":
    audio_path = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\ASVspoof2019_LA_train\\flac"
    labels_file = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\ASVspoof2019_LA_cm_protocols\\ASVspoof2019.LA.cm.train.trn.txt"
    output_feature_file = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\Features\\Training\\LA_train_lfcc_features.npy"
    output_labels_file = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\Features\\Training\\LA_train_lfcc_labels.npy"
    max_frames = 40  # Maximum number of frames to consider

    # Read labels
    labels_dict = read_labels(labels_file)

    # Extract features and store labels
    features_list = []
    labels_list = []

    # Iterate over the labeled audio files
    for audio_name, label in tqdm(labels_dict.items(), desc="Extracting features"):
        audio_file = os.path.join(audio_path, audio_name + ".flac")
        # Extract feature
        feature = extract_feature(audio_file, lfcc, max_frames)
        # Store feature and label
        features_list.append(feature)
        labels_list.append(label)
        # Print label
        # print(f"File: {audio_name}, Label: {'spoof' if label == 0 else 'bonafide'}")

    # Convert lists to numpy arrays
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    # Save features and labels to numpy files
    np.save(output_feature_file, features_array)
    np.save(output_labels_file, labels_array)

Extracting features: 100%|██████████████████████████████████████████████████████| 25380/25380 [03:51<00:00, 109.85it/s]


In [30]:
def extract_feature(audio_file, feature_func, max_frames):
    # Load audio file to get sample rate
    signal, sample_rate = sf.read(audio_file)

    # Load audio file up to 4 seconds
    signal = signal[:int(4 * sample_rate)]

    # Extract feature
    feature = feature_func(signal, fs=sample_rate)

    # Pad or truncate feature to fixed length
    if feature.shape[0] < max_frames:
        feature = np.pad(feature, ((0, max_frames - feature.shape[0]), (0, 0)), mode='constant')
    elif feature.shape[0] > max_frames:
        feature = feature[:max_frames, :]

    return feature

def read_labels(labels_file):
    labels_dict = {}
    with open(labels_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            audio_name = parts[1]
            label = parts[4]  # Assuming the label is at the 5th index
            # print(label)
            label = 1 if label == 'spoof' else 0
            labels_dict[audio_name] = label
    return labels_dict

if __name__ == "__main__":
    audio_path = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\ASVspoof2019_LA_dev\\flac"
    labels_file = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\ASVspoof2019_LA_cm_protocols\\ASVspoof2019.LA.cm.dev.trl.txt"
    output_feature_file = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\Features\\Development\\LA_dev_lfcc_features.npy"
    output_labels_file = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\Features\\Development\\LA_dev_lfcc_labels.npy"
    max_frames = 40  # Maximum number of frames to consider

    # Read labels
    labels_dict = read_labels(labels_file)

    # Extract features and store labels
    features_list = []
    labels_list = []

    # Iterate over the labeled audio files
    for audio_name, label in tqdm(labels_dict.items(), desc="Extracting features"):
        audio_file = os.path.join(audio_path, audio_name + ".flac")
        # Extract feature
        feature = extract_feature(audio_file, lfcc, max_frames)
        # Store feature and label
        features_list.append(feature)
        labels_list.append(label)
        # Print label
        # print(f"File: {audio_name}, Label: {'spoof' if label == 0 else 'bonafide'}")

    # Convert lists to numpy arrays
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    # Save features and labels to numpy files
    np.save(output_feature_file, features_array)
    np.save(output_labels_file, labels_array)

Extracting features: 100%|██████████████████████████████████████████████████████| 24844/24844 [03:38<00:00, 113.95it/s]


In [31]:
import os
import numpy as np
import soundfile as sf  # Assuming you've imported soundfile as sf
from tqdm import tqdm

# Function to extract features
def extract_feature(audio_file, feature_func, max_frames):
    if not os.path.exists(audio_file):
        print(f"File '{audio_file}' not found. Skipping...")
        return None
    
    # Load audio file to get sample rate
    signal, sample_rate = sf.read(audio_file)

    # Load audio file up to 4 seconds
    signal = signal[:int(4 * sample_rate)]

    # Extract feature
    feature = feature_func(signal, fs=sample_rate)

    # Pad or truncate feature to fixed length
    if feature.shape[0] < max_frames:
        feature = np.pad(feature, ((0, max_frames - feature.shape[0]), (0, 0)), mode='constant')
    elif feature.shape[0] > max_frames:
        feature = feature[:max_frames, :]

    return feature

# Function to read labels
def read_labels(labels_file):
    labels_dict = {}
    with open(labels_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            audio_name = parts[1]
            label = parts[4]  # Assuming the label is at the 5th index
            label = 1 if label == 'spoof' else 0
            labels_dict[audio_name] = label
    return labels_dict

if __name__ == "__main__":
    audio_path = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\ASVspoof2019_LA_eval\\flac"
    labels_file = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\ASVspoof2019_LA_cm_protocols\\ASVspoof2019.LA.cm.eval.trl.txt"
    output_feature_file = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\Features\\Evaluation\\LA_eval_lfcc_features.npy"
    output_labels_file = "F:\\Awais_data\\Datasets\\asvspoof2019\\LA\\Features\\Evaluation\\LA_eval_lfcc_labels.npy"
    max_frames = 40  # Maximum number of frames to consider

    # Read labels
    labels_dict = read_labels(labels_file)

    # Extract features and store labels
    features_list = []
    labels_list = []

    # Iterate over the labeled audio files
    for audio_name, label in tqdm(labels_dict.items(), desc="Extracting features"):
        audio_file = os.path.join(audio_path, audio_name + ".flac")
        
        # Check if file exists
        if not os.path.exists(audio_file):
            print(f"File '{audio_file}' not found. Skipping...")
            continue
        
        # Extract feature
        feature = extract_feature(audio_file, lfcc, max_frames)
        
        # Store feature and label if feature extraction was successful
        if feature is not None:
            features_list.append(feature)
            labels_list.append(label)

    # Convert lists to numpy arrays
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    # Save features and labels to numpy files
    np.save(output_feature_file, features_array)
    np.save(output_labels_file, labels_array)


Extracting features:   0%|                                                         | 61/71237 [00:00<09:59, 118.65it/s]

File 'F:\Awais_data\Datasets\asvspoof2019\LA\ASVspoof2019_LA_eval\flac\LA_E_2634822.flac' not found. Skipping...


Extracting features:   0%|                                                         | 87/71237 [00:00<09:42, 122.09it/s]

File 'F:\Awais_data\Datasets\asvspoof2019\LA\ASVspoof2019_LA_eval\flac\LA_E_5118048.flac' not found. Skipping...


Extracting features:   0%|                                                        | 116/71237 [00:00<09:03, 130.92it/s]

File 'F:\Awais_data\Datasets\asvspoof2019\LA\ASVspoof2019_LA_eval\flac\LA_E_2161075.flac' not found. Skipping...


Extracting features:   0%|▏                                                       | 201/71237 [00:01<08:51, 133.65it/s]

File 'F:\Awais_data\Datasets\asvspoof2019\LA\ASVspoof2019_LA_eval\flac\LA_E_7205247.flac' not found. Skipping...
File 'F:\Awais_data\Datasets\asvspoof2019\LA\ASVspoof2019_LA_eval\flac\LA_E_1911910.flac' not found. Skipping...


Extracting features: 100%|███████████████████████████████████████████████████████| 71237/71237 [12:03<00:00, 98.40it/s]


ASVspoof21

In [28]:
import os
import numpy as np
import soundfile as sf  # Assuming you've imported soundfile as sf
from tqdm import tqdm

# Function to extract features
def extract_feature(audio_file, feature_func, max_frames):
    if not os.path.exists(audio_file):
        print(f"File '{audio_file}' not found. Skipping...")
        return None
    
    # Load audio file to get sample rate
    signal, sample_rate = sf.read(audio_file)

    # Load audio file up to 4 seconds
    signal = signal[:int(4 * sample_rate)]

    # Extract feature
    feature = feature_func(signal, fs=sample_rate)

    # Pad or truncate feature to fixed length
    if feature.shape[0] < max_frames:
        feature = np.pad(feature, ((0, max_frames - feature.shape[0]), (0, 0)), mode='constant')
    elif feature.shape[0] > max_frames:
        feature = feature[:max_frames, :]

    return feature

# Function to read labels
def read_labels(labels_file):
    labels_dict = {}
    with open(labels_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            audio_name = parts[1]
            label = parts[5]  # Assuming the label is at the 5th index
            label = 1 if label == 'spoof' else 0
            labels_dict[audio_name] = label
    return labels_dict

if __name__ == "__main__":
    audio_path = "F:\\Awais_data\\Datasets\\ASV21\\ASVspoof2021_DF_eval\\flac"
    labels_file = "F:\\Awais_data\\Datasets\\ASV21\\ASVspoof2021_DF_eval\\ASVspoof2021.DF.cm.eval.trl.txt.txt"
    output_feature_file = "F:\\Awais_data\\Datasets\\ASV21\\Features\\DF\\DF_eval_lfcc_features.npy"
    output_labels_file = "F:\\Awais_data\\Datasets\\ASV21\\Features\\DF\\DF_eval_lfcc_labels.npy"
    max_frames = 40  # Maximum number of frames to consider

    # Read labels
    labels_dict = read_labels(labels_file)

    # Extract features and store labels
    features_list = []
    labels_list = []

    # Iterate over the labeled audio files
    for audio_name, label in tqdm(labels_dict.items(), desc="Extracting features"):
        audio_file = os.path.join(audio_path, audio_name + ".flac")
        
        # Check if file exists
        if not os.path.exists(audio_file):
            print(f"File '{audio_file}' not found. Skipping...")
            continue
        
        # Extract feature
        feature = extract_feature(audio_file, lfcc, max_frames)
        
        # Store feature and label if feature extraction was successful
        if feature is not None:
            features_list.append(feature)
            labels_list.append(label)

    # Convert lists to numpy arrays
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    # Save features and labels to numpy files
    np.save(output_feature_file, features_array)
    np.save(output_labels_file, labels_array)

Extracting features: 100%|██████████████████████████████████████████████████| 611829/611829 [22:40:14<00:00,  7.50it/s]


In [27]:
import os
import numpy as np
import soundfile as sf  # Assuming you've imported soundfile as sf
from tqdm import tqdm

# Function to extract features
def extract_feature(audio_file, feature_func, max_frames):
    if not os.path.exists(audio_file):
        print(f"File '{audio_file}' not found. Skipping...")
        return None
    
    # Load audio file to get sample rate
    signal, sample_rate = sf.read(audio_file)

    # Load audio file up to 4 seconds
    signal = signal[:int(4 * sample_rate)]

    # Extract feature
    feature = feature_func(signal, fs=sample_rate)

    # Pad or truncate feature to fixed length
    if feature.shape[0] < max_frames:
        feature = np.pad(feature, ((0, max_frames - feature.shape[0]), (0, 0)), mode='constant')
    elif feature.shape[0] > max_frames:
        feature = feature[:max_frames, :]

    return feature

# Function to read labels
def read_labels(labels_file):
    labels_dict = {}
    with open(labels_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            audio_name = parts[1]
            label = parts[5]  # Assuming the label is at the 5th index
            label = 1 if label == 'spoof' else 0
            labels_dict[audio_name] = label
    return labels_dict

if __name__ == "__main__":
    audio_path = "F:\\Awais_data\\Datasets\\ASV21\\ASVspoof2021_LA_eval\\flac"
    labels_file = "F:\\Awais_data\\Datasets\\ASV21\\ASVspoof2021_LA_eval\\ASVspoof2021.LA.cm.eval.trl.txt.txt"
    output_feature_file = "F:\\Awais_data\\Datasets\\ASV21\\Features\\LA\\LA_eval_mfcc_features.npy"
    output_labels_file = "F:\\Awais_data\\Datasets\\ASV21\\Features\\LA\\LA_eval_mfcc_labels.npy"
    max_frames = 40  # Maximum number of frames to consider

    # Read labels
    labels_dict = read_labels(labels_file)

    # Extract features and store labels
    features_list = []
    labels_list = []

    # Iterate over the labeled audio files
    for audio_name, label in tqdm(labels_dict.items(), desc="Extracting features"):
        audio_file = os.path.join(audio_path, audio_name + ".flac")
        
        # Check if file exists
        if not os.path.exists(audio_file):
            print(f"File '{audio_file}' not found. Skipping...")
            continue
        
        # Extract feature
        feature = extract_feature(audio_file, mfcc, max_frames)
        
        # Store feature and label if feature extraction was successful
        if feature is not None:
            features_list.append(feature)
            labels_list.append(label)

    # Convert lists to numpy arrays
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    # Save features and labels to numpy files
    np.save(output_feature_file, features_array)
    np.save(output_labels_file, labels_array)

Extracting features: 100%|████████████████████████████████████████████████████| 181566/181566 [25:32<00:00, 118.47it/s]


WaveFake

In [25]:
# Function to extract features
def extract_feature(audio_file, feature_func, max_frames):
    if not os.path.exists(audio_file):
        print(f"File '{audio_file}' not found. Skipping...")
        return None
    
    # Load audio file to get sample rate
    signal, sample_rate = sf.read(audio_file)
    signal = signal[:int(4 * sample_rate)]
    feature = feature_func(signal, fs=sample_rate)

    # Pad or truncate feature to fixed length
    if feature.shape[0] < max_frames:
        feature = np.pad(feature, ((0, max_frames - feature.shape[0]), (0, 0)), mode='constant')
    elif feature.shape[0] > max_frames:
        feature = feature[:max_frames, :]
    return feature

# Function to process audio files in directories
def process_directory(directory, feature_func, max_frames):
    features_list = []
    labels_list = []
    
    for root, _, files in os.walk(directory):
        for file in tqdm(files, desc=f"Processing {root}"):
            if file.endswith(".wav"):
                audio_file = os.path.join(root, file)
                feature = extract_feature(audio_file, feature_func, max_frames)
                
                if feature is not None:
                    features_list.append(feature)
                    labels_list.append(1)  # All labels are set to 1 (spoof)
    
    return np.array(features_list), np.array(labels_list)

if __name__ == "__main__":
    base_dir = "F:\\Awais_data\\Datasets\\wavefake_dataset\\generated_audio"
    ljspeech_dirs = [
        "ljspeech_common_voices_prompts_from_conformer_fastspeech2",
        "ljspeech_full_band_melgan","ljspeech_hifiGAN","ljspeech_melgan","ljspeech_melgan_large",
        "ljspeech_multi_band_melgan","ljspeech_parallel_wavegan","ljspeech_waveglow"
    ]
    jsut_dirs = [
        "jsut_multi_band_melgan",
        "jsut_parallel_wavegan"
    ]

    output_feature_file_ljspeech = "F:\\Awais_data\\Datasets\\wavefake_dataset\\generated_audio\\Features\\ljspeech\\ljspeech_features.npy"
    output_labels_file_ljspeech = "F:\\Awais_data\\Datasets\\wavefake_dataset\\generated_audio\\Features\\ljspeech\\ljspeech_labels.npy"
    output_feature_file_jsut = "F:\\Awais_data\\Datasets\\wavefake_dataset\\generated_audio\\Features\\jsut\\jsut_mfcc_features.npy"
    output_labels_file_jsut = "F:\\Awais_data\\Datasets\\wavefake_dataset\\generated_audio\\Features\\jsut\\jsut_mfcc_labels.npy"
    max_frames = 40  # Maximum number of frames to consider

    
    # Process ljspeech directories
    ljspeech_features_list = []
    ljspeech_labels_list = []
    for ljspeech_dir in ljspeech_dirs:
        dir_path = os.path.join(base_dir, ljspeech_dir)
        features, labels = process_directory(dir_path, lfcc, max_frames)
        ljspeech_features_list.append(features)
        ljspeech_labels_list.append(labels)
    
    ljspeech_features = np.vstack(ljspeech_features_list)
    ljspeech_labels = np.hstack(ljspeech_labels_list)

    # Save ljspeech features and labels
    np.save(output_feature_file_ljspeech, ljspeech_features)
    np.save(output_labels_file_ljspeech, ljspeech_labels)

    # Process jsut directories
    jsut_features_list = []
    jsut_labels_list = []
    for jsut_dir in jsut_dirs:
        dir_path = os.path.join(base_dir, jsut_dir)
        features, labels = process_directory(dir_path, mfcc, max_frames)
        jsut_features_list.append(features)
        jsut_labels_list.append(labels)
    
    jsut_features = np.vstack(jsut_features_list)
    jsut_labels = np.hstack(jsut_labels_list)

    # Save jsut features and labels
    np.save(output_feature_file_jsut, jsut_features)
    np.save(output_labels_file_jsut, jsut_labels)


Processing F:\Awais_data\Datasets\wavefake_dataset\generated_audio\ljspeech_common_voices_prompts_from_conformer_fastsp
Processing F:\Awais_data\Datasets\wavefake_dataset\generated_audio\ljspeech_full_band_melgan: 100%|█| 13100/13100 [02:0
Processing F:\Awais_data\Datasets\wavefake_dataset\generated_audio\ljspeech_hifiGAN: 100%|█| 13100/13100 [02:12<00:00, 
Processing F:\Awais_data\Datasets\wavefake_dataset\generated_audio\ljspeech_melgan: 100%|█| 13100/13100 [02:09<00:00, 1
Processing F:\Awais_data\Datasets\wavefake_dataset\generated_audio\ljspeech_melgan_large: 100%|█| 13100/13100 [02:11<00
Processing F:\Awais_data\Datasets\wavefake_dataset\generated_audio\ljspeech_multi_band_melgan: 100%|█| 13100/13100 [02:
Processing F:\Awais_data\Datasets\wavefake_dataset\generated_audio\ljspeech_parallel_wavegan: 100%|█| 13099/13099 [02:0
Processing F:\Awais_data\Datasets\wavefake_dataset\generated_audio\ljspeech_waveglow: 100%|█| 13100/13100 [02:08<00:00,
Processing F:\Awais_data\Datasets\wavefa